# Explorer Jesus

In [2]:
# Your code here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
data = pd.read_csv('/workspaces/Final_Project_DataScient/data/interim/data_limpia.zip')
df = data.copy()
df

,MARITAL,RENTHOM1,VETERAN3,EMPLOY1,_AGEG5YR,_INCOMG1,_EDUCAG,_CHLDCNT,_EXTETH3,HHADULT,...,ACEPRISN,ACEDIVRC,ACEHURT1,ACESWEAR,ACETOUCH,ACETTHEM,ACEHVSEX,FIREARM5,GUNLOAD,LOADULK2
0,1.0,1.0,2.0,7.0,13.0,9.0,4.0,1.0,9.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3.0,1.0,2.0,2.0,13.0,3.0,2.0,1.0,9.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,1.0,2.0,7.0,8.0,6.0,4.0,1.0,9.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,1.0,2.0,7.0,14.0,9.0,2.0,1.0,9.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,1.0,2.0,5.0,5.0,3.0,3.0,1.0,9.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445127,5.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
445128,1.0,1.0,1.0,1.0,7.0,5.0,4.0,2.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
445129,1.0,3.0,2.0,8.0,10.0,9.0,2.0,1.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
445130,1.0,2.0,2.0,7.0,11.0,5.0,3.0,1.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
columnas_a_borrar = []

for i in data.columns:
    porcentaje_nulos = df[i].isnull().sum() / len(df) * 100
    print(f'Columna: {i} - Porcentaje de valores nulos: {porcentaje_nulos:.2f}%')
    if porcentaje_nulos >= 50:
        columnas_a_borrar.append(i)

Columna: MARITAL - Porcentaje de valores nulos: 0.00%
Columna: RENTHOM1 - Porcentaje de valores nulos: 0.00%
Columna: VETERAN3 - Porcentaje de valores nulos: 0.94%
Columna: EMPLOY1 - Porcentaje de valores nulos: 1.39%
Columna: _AGEG5YR - Porcentaje de valores nulos: 0.00%
Columna: _INCOMG1 - Porcentaje de valores nulos: 0.00%
Columna: _EDUCAG - Porcentaje de valores nulos: 0.00%
Columna: _CHLDCNT - Porcentaje de valores nulos: 0.00%
Columna: _EXTETH3 - Porcentaje de valores nulos: 0.00%
Columna: HHADULT - Porcentaje de valores nulos: 21.58%
Columna: SEXVAR - Porcentaje de valores nulos: 0.00%
Columna: PERSDOC3 - Porcentaje de valores nulos: 0.00%
Columna: MEDCOST1 - Porcentaje de valores nulos: 0.00%
Columna: _HLTHPLN - Porcentaje de valores nulos: 0.00%
Columna: _AIDTST4 - Porcentaje de valores nulos: 10.39%
Columna: SDHEMPLY - Porcentaje de valores nulos: 43.05%
Columna: SDHFOOD1 - Porcentaje de valores nulos: 43.20%
Columna: SDHUTILS - Porcentaje de valores nulos: 43.32%
Columna: SD

In [8]:
columnas_a_borrar = []

for i in df.columns:
    porcentaje_nulos = df[i].isnull().sum() / len(df) * 100
    print(f'Columna: {i} - Porcentaje de valores nulos: {porcentaje_nulos:.2f}%')
    if porcentaje_nulos >= 60:
        columnas_a_borrar.append(i)

Columna: MARITAL - Porcentaje de valores nulos: 0.00%
Columna: RENTHOM1 - Porcentaje de valores nulos: 0.00%
Columna: VETERAN3 - Porcentaje de valores nulos: 0.94%
Columna: EMPLOY1 - Porcentaje de valores nulos: 1.39%
Columna: _AGEG5YR - Porcentaje de valores nulos: 0.00%
Columna: _INCOMG1 - Porcentaje de valores nulos: 0.00%
Columna: _EDUCAG - Porcentaje de valores nulos: 0.00%
Columna: _CHLDCNT - Porcentaje de valores nulos: 0.00%
Columna: _EXTETH3 - Porcentaje de valores nulos: 0.00%
Columna: HHADULT - Porcentaje de valores nulos: 21.58%
Columna: SEXVAR - Porcentaje de valores nulos: 0.00%
Columna: PERSDOC3 - Porcentaje de valores nulos: 0.00%
Columna: MEDCOST1 - Porcentaje de valores nulos: 0.00%
Columna: _HLTHPLN - Porcentaje de valores nulos: 0.00%
Columna: _AIDTST4 - Porcentaje de valores nulos: 10.39%
Columna: SDHEMPLY - Porcentaje de valores nulos: 43.05%
Columna: SDHFOOD1 - Porcentaje de valores nulos: 43.20%
Columna: SDHUTILS - Porcentaje de valores nulos: 43.32%
Columna: SD

In [9]:
columnas_a_borrar

['CDSOCIAL',
 'CDDISCUS',
 'CAREGIV1',
 'CRGVREL4',
 'CRGVLNG1',
 'CRGVHRS1',
 'MARIJAN1',
 'MARJSMOK',
 'MARJEAT',
 'MARJVAPE',
 'MARJDAB',
 'MARJOTHR',
 'ASBIALCH',
 'ASBIDRNK',
 'ASBIBING',
 'ACEDEPRS',
 'ACEDRINK',
 'ACEDRUGS',
 'ACEPRISN',
 'ACEDIVRC',
 'ACEHURT1',
 'ACESWEAR',
 'ACETOUCH',
 'ACETTHEM',
 'ACEHVSEX',
 'FIREARM5',
 'GUNLOAD',
 'LOADULK2']

### Columnas a modificar por Jesús

In [ ]:
# Question:  Adults who reported doing physical activity or exercise during the past 30 days other than their regular job
df['_TOTINDA'].value_counts()

_TOTINDA
1.0    337559
2.0    106480
9.0      1093
Name: count, dtype: int64

In [ ]:
# 9 (Don’t know/Refused/Missing) >> Cambiar a -1
df.loc[df["_TOTINDA"] == 9, "_TOTINDA"] = -1
df.loc[df["_TOTINDA"] == 2, "_TOTINDA"] = 0

df["_TOTINDA"].value_counts()

_TOTINDA
 1.0    337559
 0.0    106480
-1.0      1093
Name: count, dtype: int64

In [ ]:
# Question:  (Ever told) (you had) a depressive disorder (including depression, major depression, dysthymia, or minor depression)
df['ADDEPEV3'].value_counts(dropna=False)

ADDEPEV3
 0.0    350910
 1.0     91410
-1.0      2805
 NaN         7
Name: count, dtype: int64

In [ ]:
# Eliminamos NAN y los valores 7 y 9 (Don’t know/Refused/Missing)
# Pasamos 2 (no) a 0

df['ADDEPEV3'].dropna()
df = df[~df["ADDEPEV3"].isin([-1])]
df.loc[df["ADDEPEV3"] == 2, "ADDEPEV3"] = 0


print(df["ADDEPEV3"].value_counts())
print(df.shape)

ADDEPEV3
0.0    350910
1.0     91410
Name: count, dtype: int64
(442327, 35)


In [ ]:
# Question:  Has a doctor, nurse, or other health professional ever told you that you tested positive for COVID 19?
df['COVIDPOS'].value_counts(dropna=False)

COVIDPOS
2.0    268442
1.0    110273
NaN     48851
3.0     13375
9.0       759
7.0       627
Name: count, dtype: int64